In [1]:
# system import
import sys
import os
import csv
import config
import random
import time

# data processing
import pandas as pd


# data Extraction
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests

In [2]:
# set env
PROJECT_PATH = config.data_directory
client_credentials_manager = SpotifyClientCredentials(client_id=config.client_id, client_secret=config.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# try to not get banned
USER_AGENTS = [
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Mozilla/5.0 (iPad; CPU OS 8_4_1 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12H321 Safari/600.1.4',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393'
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.7; rv:11.0) Gecko/20100101 Firefox/11.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36 OPR/58.0.3135.107']

def getFromUrl(url, user_agent=True):
    time.sleep(random.randint(1,8))
    if user_agent:
        return requests.get(url, headers={'User-Agent': random.choice(USER_AGENTS)})
    return requests.get(url)

def getLyric(title):
    url = "https://www.azlyrics.com/lyrics/{}.html".format(title)
    return getFromUrl(url)

def seachLyric(title, artist):
    url = "https://search.azlyrics.com/search.php?q={}".format(title.replace(' ', '+').lower())
    c = getFromUrl(url)
    soup = BeautifulSoup(c.content)
    links = soup.find_all('b', string=artist)
    if len(links) > 0:
        link = links[0].find_previous_sibling('a')['href']
        return getFromUrl(link)
    else:
        return None

In [3]:
# file operations
def read_file(file):
    text_file = open('{}'.format(file), "r")
    text = text_file.read()
    text_file.close()
    return text

def get_lyric_name_path(a, t):
    aaz = a.replace(' ', '').lower()
    taz = t.replace(' ', '').lower()
    title = '{}/{}'.format(aaz, taz)
    print(title)
    file_path = '{}/{}.txt'.format(PROJECT_PATH, title.replace('/','-'))
    path = [title, file_path]

    if not os.path.exists(file_path):
        return path + [True]
    else:
        return path + [False]

def fetch_lylic_to_file(a, t, aa = None, tt = None):
    path = get_lyric_name_path(a, t)

    if not path[2]:
        result = getLyric(path[0])
        
        if result.status_code != requests.codes.ok:
            result = seachLyric(t, aa)
            
        if result is None:
            return False
            
        c = result.content
        soup = BeautifulSoup(c)
        ringtones = soup.find_all("div", class_="ringtone")
        
        if len(ringtones) > 0:
            ringtone = ringtones[0]
            html_lyrics = ringtone.find_next_sibling('div')
            cleanText = html_lyrics.get_text().replace('\r','').replace('\n',' ').replace("\'",'"')

            fo = open(path[1], "w", encoding='utf-8')
            fo.write(cleanText)
            fo.close()
            
            return True
        else:
            return False
    else:
        print('File Already Exists')
        return False

In [4]:
def splitChums(target, n = 100):
    x = (len(target) + n - 1) // n
    rg = range(x)
    return [target[i * n:(i + 1) * n] for i in rg]

In [5]:
# spotify interaction
def fetch_songs(offset, count, limit = 50, startYear = 1990, name = 'songs'):
    file_path = '{}/{}.csv'.format(PROJECT_PATH, name)
    new_file = 'a'
    tracks = []
    if not os.path.exists(file_path):
        tracks = [['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artist', 'artist_id', 'album', 'album_id', 'album_release_date', 'album_total_tracks']]
        new_file = 'w'

    partition = (((count * limit) + 10 - 1) // 10)
    with open(file_path, new_file, newline = '', encoding='utf-8') as csvFile:
        writer = csv.writer(csvFile)
        for i in range(offset, count * limit, limit):
            if i % partition: 
                startYear = startYear + 1
            track_results = sp.search(q='year:' + str(startYear), type='track', limit=limit, offset=i)
            for i, t in enumerate(track_results['tracks']['items']):
                tracks.append([t['id'], t['name'], t['popularity'], t['duration_ms'], t['explicit'], t['artists'][0]['name'], t['artists'][0]['id'],  t['album']['name'], t['album']['id'], t['album']['release_date'], t['album']['total_tracks']])
        writer.writerows(tracks)
    csvFile.close()

In [6]:
fetch_songs(0, 10)

In [7]:
# load dataFrame
df = pd.read_csv('{}/{}.csv'.format(PROJECT_PATH, 'songs'))

# az data
pattern = r'[^a-z0-9A-z]'
df['p_name'] = df['name'].str.replace(pattern, ' ').str.lower()
df['p_artist'] = df['artist'].str.replace(pattern, ' ').str.lower()
df['file_name'] = df.apply(lambda x: get_lyric_name_path(x['p_artist'], x['p_name'])[1], axis=1)
df['has_lyrics'] = False

acdc/thunderstruck
eltonjohn/dontgobreakingmyheart
aliceinchains/maninthebox
bellbivdevoe/poison
theymightbegiants/istanbul
atribecalledquest/canikickit
extreme/morethanwords
theromantics/whatilikeaboutyou
pantera/cowboysfromhell
mchammer/ucanttouchthis
theblackcrowes/hardtohandle
warrant/cherrypie
digitalunderground/thehumptydance
wilsonphillips/holdon
joediffie/johndeeregreen
thelas/thereshegoes
scorpions/windofchange
depechemode/enjoythesilencesinglemix
ramjam/blackbetty
joediffie/pickupman
therighteousbrothers/unchainedmelody
beegees/stayinalivefromsaturdaynightfeversoundtrack
templeofthedog/hungerstrike
sinadoconnor/nothingcompares2u
pantera/cemeterygates
keithwhitley/dontcloseyoureyes
juangabriel/abrzamemuyfuerte
man/rayandoelsol
joediffie/propmeupbesidethejukeboxifidie
megadeth/holywarsthepunishmentdueremastered2004
llcoolj/mamasaidknockyouout
joediffie/pickupman
tonytoniton/feelsgood
vanillaice/iceicebaby
poison/unskinnybop
heart/alliwannadoismakelovetoyou
jonbonjovi/blazeofglo

In [8]:
# get audio features
ids = df['id'].tolist()

cIds = splitChums(ids)

features = []

for cId in cIds:
    audio_results = sp.audio_features(cId)
    for af in audio_results:
        if af:
            features.append([af['id'], af['danceability'], af['energy'], af['key'], af['loudness'], af['mode'], af['speechiness'], af['acousticness'], af['instrumentalness'], af['liveness'], af['valence'], af['tempo']])

In [9]:
# get audio features
ids = df['artist_id'].tolist()
ids = list(dict.fromkeys(ids))
cIds = splitChums(ids, 50)
artists = []

for cId in cIds:
    artists_results = sp.artists(cId)
    for ar in artists_results['artists']:
        if ar:
            artists.append([ar['id'], ar['genres']])

In [10]:
dfaf = pd.DataFrame(features, columns = ['id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'])
dfa = pd.DataFrame(artists, columns = ['artist_id', 'genres'])

df = pd.merge(df, dfaf, on='id', how='inner')
df = pd.merge(df, dfa, on='artist_id', how='inner')

In [11]:
# save dataframe to csv file
def dataframe_to_csv(pDf, name = 'songs'):
    file_path = '{}/{}.csv'.format(PROJECT_PATH, name)
    pDf.to_csv(file_path, sep=',', encoding='utf-8', index=False)

In [ ]:
# fetch from internet
for index, row in df.iterrows():
    df.loc[index, 'has_lyrics'] = fetch_lylic_to_file(row['p_artist'], row['p_name'], row['artist'], row['name'])

In [12]:
# fetch from local
for index, row in df.iterrows():
    exists = os.path.isfile(row['file_name'])
    if exists:
        df.loc[index, 'has_lyrics'] = exists

In [ ]:
df.tail(50)

In [13]:
dataframe_to_csv(df)